In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, confusion_matrix, classification_report
import lightgbm as lgb
import xgboost as xgb
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 1. Chargement et préparation des données
print("1. Chargement et préparation des données")
data = pd.read_csv('df_merged_bretagne_filtre.csv', sep=';', decimal=',')

# Conversion des colonnes numériques
numeric_cols = ['Votants', '% Votants', '%_voix_obtenu',
                '0 à 19 ans', '20 à 39 ans', '40 à 59 ans', '60 à 74 ans', '75 ans et plus',
                'Total', 'taux_chomage', 'taux_pour_mille', 'nombre_entreprises',
                'Baccalauréat professionnel', 'Baccalauréat technologique', 'Baccalauréat général']

for col in numeric_cols:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Création de nouvelles variables
data['% 0-19 ans'] = data['0 à 19 ans'] / data['Total'] * 100
data['% 20-39 ans'] = data['20 à 39 ans'] / data['Total'] * 100
data['% 40-59 ans'] = data['40 à 59 ans'] / data['Total'] * 100
data['% 60-74 ans'] = data['60 à 74 ans'] / data['Total'] * 100
data['% 75+ ans'] = data['75 ans et plus'] / data['Total'] * 100

# Création d'une variable binaire pour la victoire (1 si le candidat a obtenu plus de 50% des voix)
data['victoire'] = (data['%_voix_obtenu'] > 50).astype(int)

# Affichage des premières lignes du dataset préparé
print("Aperçu des données préparées:")
print(data.head())
print("\nDimensions du dataset:", data.shape)

# 2. Analyse de la distribution des variables
print("\n2. Analyse de la distribution des variables")
print("Distribution des résultats électoraux par orientation politique:")
print(data.groupby('COULEUR POLITIQUE')['%_voix_obtenu'].describe())

# 3. Préparation des données pour la modélisation
print("\n3. Préparation des données pour la modélisation")

# Variables explicatives et cible
# Pour la prédiction du pourcentage de voix (régression)
X_reg = data[['Année', 'taux_chomage', 'nombre_entreprises',
          'Baccalauréat professionnel', 'Baccalauréat technologique', 'Baccalauréat général',
          '% 0-19 ans', '% 20-39 ans', '% 40-59 ans', '% 60-74 ans', '% 75+ ans',
          '% Votants']]
y_reg = data['%_voix_obtenu']

# Pour la prédiction de la victoire (classification)
X_clf = X_reg.copy()
y_clf = data['victoire']

# Séparation en ensembles d'entraînement et de test (80% / 20%)
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)

# 4. Entraînement du modèle de régression (XGBoost)
print("\n4. Entraînement du modèle de régression XGBoost")

# Modèle XGBoost pour la régression
model_xgb_reg = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    random_state=42
)

# Entraînement du modèle
model_xgb_reg.fit(X_reg_train, y_reg_train)

# Prédictions sur l'ensemble de test
y_reg_pred = model_xgb_reg.predict(X_reg_test)

# Évaluation du modèle de régression
mae = mean_absolute_error(y_reg_test, y_reg_pred)
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
r2 = r2_score(y_reg_test, y_reg_pred)

print(f"Résultats du modèle de régression (prédiction du % de voix):")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")

# 5. Entraînement du modèle de classification (LightGBM)
print("\n5. Entraînement du modèle de classification LightGBM")

# Modèle LightGBM pour la classification
model_lgb_clf = lgb.LGBMClassifier(
    objective='binary',
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    random_state=42
)

# Entraînement du modèle
model_lgb_clf.fit(X_clf_train, y_clf_train)

# Prédictions sur l'ensemble de test
y_clf_pred = model_lgb_clf.predict(X_clf_test)
y_clf_prob = model_lgb_clf.predict_proba(X_clf_test)[:, 1]

# Évaluation du modèle de classification
print("Matrice de confusion:")
cm = confusion_matrix(y_clf_test, y_clf_pred)
print(cm)
print("\nRapport de classification:")
print(classification_report(y_clf_test, y_clf_pred))

# 6. Analyse de l'importance des variables
print("\n6. Analyse de l'importance des variables")

# Importance des variables pour le modèle de régression
feature_importance_reg = model_xgb_reg.feature_importances_
feature_names = X_reg.columns
features_df_reg = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance_reg
}).sort_values(by='Importance', ascending=False)

print("Importance des variables pour la prédiction du pourcentage de voix:")
print(features_df_reg)

# Importance des variables pour le modèle de classification
feature_importance_clf = model_lgb_clf.feature_importances_
features_df_clf = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance_clf
}).sort_values(by='Importance', ascending=False)

print("\nImportance des variables pour la prédiction de la victoire:")
print(features_df_clf)

# 7. Visualisation des prédictions
print("\n7. Visualisation des prédictions")

# Visualisation 1: Prédictions vs valeurs réelles pour le modèle de régression
fig1 = px.scatter(
    x=y_reg_test,
    y=y_reg_pred,
    labels={'x': 'Pourcentage réel de voix', 'y': 'Pourcentage prédit de voix'},
    title="Prédictions vs Valeurs Réelles (Pourcentage de Voix)",
    trendline='ols'
)
fig1.add_shape(
    type='line', line=dict(dash='dash', color='black'),
    x0=y_reg_test.min(), y0=y_reg_test.min(),
    x1=y_reg_test.max(), y1=y_reg_test.max()
)
fig1.update_layout(
    width=800,
    height=600,
    font=dict(size=12)
)

# Visualisation 2: Importance des variables pour le modèle de régression
fig2 = px.bar(
    features_df_reg,
    x='Importance',
    y='Feature',
    orientation='h',
    title="Importance des Variables pour la Prédiction du Pourcentage de Voix",
    color='Importance',
    color_continuous_scale='viridis'
)
fig2.update_layout(
    width=800,
    height=600,
    yaxis={'categoryorder': 'total ascending'},
    font=dict(size=12)
)

# Visualisation 3: Résidus du modèle de régression
residus = y_reg_test - y_reg_pred
fig3 = px.scatter(
    x=y_reg_pred,
    y=residus,
    labels={'x': 'Prédictions', 'y': 'Résidus'},
    title="Analyse des Résidus du Modèle de Régression"
)
fig3.add_shape(
    type='line', line=dict(dash='dash', color='red'),
    x0=y_reg_pred.min(), y0=0,
    x1=y_reg_pred.max(), y1=0
)
fig3.update_layout(
    width=800,
    height=600,
    font=dict(size=12)
)

# Visualisation 4: Courbe ROC pour le modèle de classification
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_clf_test, y_clf_prob)
roc_auc = auc(fpr, tpr)

fig4 = px.area(
    x=fpr, y=tpr,
    title=f'Courbe ROC (AUC = {roc_auc:.2f})',
    labels=dict(x='Taux de faux positifs', y='Taux de vrais positifs'),
    width=700, height=500
)
fig4.add_shape(
    type='line', line=dict(dash='dash', color='gray'),
    x0=0, y0=0,
    x1=1, y1=1
)
fig4.update_layout(
    width=800,
    height=600,
    font=dict(size=12)
)

# Visualisation 5: Projection des tendances électorales sur les prochaines années
# Créer un DataFrame pour les projections futures
years_future = [2027, 2032]
future_data = pd.DataFrame({
    'Année': years_future,
    'taux_chomage': [data['taux_chomage'].iloc[-1]] * len(years_future),  # Utilisation des dernières valeurs connues
    'nombre_entreprises': [data['nombre_entreprises'].iloc[-1] * 1.1, data['nombre_entreprises'].iloc[-1] * 1.2],  # Projection avec croissance
    'Baccalauréat professionnel': [data['Baccalauréat professionnel'].iloc[-1]] * len(years_future),
    'Baccalauréat technologique': [data['Baccalauréat technologique'].iloc[-1]] * len(years_future),
    'Baccalauréat général': [data['Baccalauréat général'].iloc[-1]] * len(years_future),
    '% 0-19 ans': [data['% 0-19 ans'].iloc[-1] * 0.98, data['% 0-19 ans'].iloc[-1] * 0.96],  # Projection avec décroissance
    '% 20-39 ans': [data['% 20-39 ans'].iloc[-1] * 0.99, data['% 20-39 ans'].iloc[-1] * 0.98],
    '% 40-59 ans': [data['% 40-59 ans'].iloc[-1]] * len(years_future),
    '% 60-74 ans': [data['% 60-74 ans'].iloc[-1] * 1.02, data['% 60-74 ans'].iloc[-1] * 1.04],  # Projection avec croissance
    '% 75+ ans': [data['% 75+ ans'].iloc[-1] * 1.03, data['% 75+ ans'].iloc[-1] * 1.06],
    '% Votants': [data['% Votants'].iloc[-1] * 0.98, data['% Votants'].iloc[-1] * 0.96]  # Projection avec décroissance
})

# Prédictions pour les projections
future_votes_pred = model_xgb_reg.predict(future_data)
future_victory_prob = model_lgb_clf.predict_proba(future_data)[:, 1]

# Combiner les données historiques et les projections
historical_years = data['Année'].unique()
historical_votes_gauche = data[data['COULEUR POLITIQUE'].isin(['Gauche', 'Centre, progressiste'])].groupby('Année')['%_voix_obtenu'].first().reindex(historical_years)
historical_votes_droite = data[data['COULEUR POLITIQUE'].isin(['Droite républicaine', 'Extrême droite'])].groupby('Année')['%_voix_obtenu'].first().reindex(historical_years)

# Créer le graphique de projection
fig5 = go.Figure()

# Ajouter les données historiques
fig5.add_trace(go.Scatter(
    x=historical_years,
    y=historical_votes_gauche,
    mode='lines+markers',
    name='Gauche/Centre-progressiste (historique)',
    line=dict(color='blue', width=2)
))

fig5.add_trace(go.Scatter(
    x=historical_years,
    y=historical_votes_droite,
    mode='lines+markers',
    name='Droite/Extrême droite (historique)',
    line=dict(color='red', width=2)
))

# Ajouter les projections
fig5.add_trace(go.Scatter(
    x=years_future,
    y=future_votes_pred,
    mode='lines+markers',
    name='Projection pourcentage de voix',
    line=dict(color='green', width=2, dash='dash'),
    marker=dict(size=10)
))

# Ajouter les zones de confiance (marges d'erreur)
fig5.add_trace(go.Scatter(
    x=years_future + years_future[::-1],
    y=list(future_votes_pred + rmse) + list((future_votes_pred - rmse)[::-1]),
    fill='toself',
    fillcolor='rgba(0,128,0,0.2)',
    line=dict(color='rgba(0,128,0,0)'),
    hoverinfo='skip',
    showlegend=False
))

fig5.update_layout(
    title='Projection des Tendances Électorales en Bretagne',
    xaxis_title='Année',
    yaxis_title='Pourcentage de Voix (%)',
    width=900,
    height=600,
    font=dict(size=12),
    xaxis=dict(
        tickmode='array',
        tickvals=list(historical_years) + years_future
    )
)

# Visualisation 6: Graphique de la probabilité de victoire pour les projections
fig6 = px.bar(
    x=years_future,
    y=future_victory_prob * 100,
    labels={'x': 'Année', 'y': 'Probabilité de victoire (%)'},
    title="Probabilité de Victoire pour les Futures Élections",
    color=future_victory_prob,
    color_continuous_scale='RdYlGn'
)
fig6.update_layout(
    width=800,
    height=600,
    font=dict(size=12),
    yaxis=dict(range=[0, 100])
)

# 8. Simulation de scénarios
print("\n8. Simulation de scénarios")

# Définition des scénarios
scenarios = pd.DataFrame({
    'Scénario': ['Optimiste', 'Pessimiste', 'Stagnation'],
    'Année': [2027] * 3,
    'taux_chomage': [4.5, 8.0, 5.8],
    'nombre_entreprises': [45000, 35000, 39000],
    'Baccalauréat professionnel': [30.0, 25.0, 27.4],
    'Baccalauréat technologique': [15.0, 16.0, 15.3],
    'Baccalauréat général': [42.0, 38.0, 40.0],
    '% 0-19 ans': [22.0, 21.0, 22.7],
    '% 20-39 ans': [21.0, 20.0, 21.5],
    '% 40-59 ans': [25.0, 25.0, 25.7],
    '% 60-74 ans': [20.0, 21.0, 19.2],
    '% 75+ ans': [12.0, 13.0, 10.8],
    '% Votants': [80.0, 72.0, 77.0]
})

# Prédictions pour les scénarios
X_scenarios = scenarios.drop(['Scénario'], axis=1)
scenarios['% de voix prédit'] = model_xgb_reg.predict(X_scenarios)
scenarios['Probabilité de victoire'] = model_lgb_clf.predict_proba(X_scenarios)[:, 1] * 100

print("Résultats des simulations de scénarios:")
print(scenarios[['Scénario', '% de voix prédit', 'Probabilité de victoire']])

# Visualisation 7: Comparaison des scénarios
fig7 = px.bar(
    scenarios,
    x='Scénario',
    y='% de voix prédit',
    color='Probabilité de victoire',
    color_continuous_scale='RdYlGn',
    title="Prédictions pour Différents Scénarios (2027)",
    labels={'% de voix prédit': 'Pourcentage de voix prédit (%)'}
)
fig7.update_layout(
    width=800,
    height=600,
    font=dict(size=12),
    yaxis=dict(range=[0, 100])
)

# 9. Conclusions et recommandations
print("\n9. Conclusions et recommandations")
print("""
Principales conclusions du modèle prédictif:
1. Les variables démographiques, en particulier les tranches d'âge 60-74 ans et 75+ ans, ont un impact significatif sur les résultats électoraux
2. Le niveau d'éducation (notamment le pourcentage de bacheliers généraux) est fortement corrélé aux tendances de vote
3. Le taux de chômage et le nombre d'entreprises sont des indicateurs économiques clés pour prédire les tendances électorales
4. La participation électorale a un impact direct sur les résultats, avec une tendance à la baisse qui pourrait modifier l'équilibre politique

Recommandations:
1. Surveiller de près l'évolution démographique et ses implications sur les comportements électoraux
2. Analyser les corrélations entre politiques éducatives et tendances électorales
3. Intégrer des données économiques plus détaillées pour améliorer la précision des prédictions
4. Développer des stratégies pour anticiper l'impact du vieillissement de la population sur les résultats électoraux
""")

# Affichage des visualisations
print("\nGénération des visualisations...")
fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()
fig6.show()
fig7.show()

1. Chargement et préparation des données
Aperçu des données préparées:
   Année    Votants  % Votants  %_voix_obtenu         nom_prenom  \
0   1995  1740335.0      82.91          50.60     Jacques CHIRAC   
1   1995  1740335.0      82.91          49.40      Lionel JOSPIN   
2   2002  1811287.0      83.03          88.56     Jacques CHIRAC   
3   2002  1811287.0      83.03          11.44  Jean-Marie LE PEN   
4   2007  2025199.0      87.53          47.38    Nicolas SARKOZY   

                           PARTI POLITIQUE    COULEUR POLITIQUE  0 à 19 ans  \
0   Rassemblement pour la République (RPR)  Droite républicaine    735549.0   
1                    Parti Socialiste (PS)               Gauche    735549.0   
2   Rassemblement pour la République (RPR)  Droite républicaine    738672.0   
3                      Front National (FN)       Extrême droite    738672.0   
4  Union pour un Mouvement Populaire (UMP)  Droite républicaine    762853.0   

   20 à 39 ans  40 à 59 ans  ...  nombre_entr

In [2]:
data.head()

,Année,Votants,% Votants,%_voix_obtenu,nom_prenom,PARTI POLITIQUE,COULEUR POLITIQUE,0 à 19 ans,20 à 39 ans,40 à 59 ans,...,nombre_entreprises,Baccalauréat professionnel,Baccalauréat technologique,Baccalauréat général,% 0-19 ans,% 20-39 ans,% 40-59 ans,% 60-74 ans,% 75+ ans,victoire
0,1995,1740335.0,82.91,50.60,Jacques CHIRAC,Rassemblement pour la République (RPR),Droite républicaine,735549.0,805794.0,665278.0,...,-4387.26,7.9,17.6,37.2,25.893413,28.366236,23.419674,15.810440,6.510237,1
1,1995,1740335.0,82.91,49.40,Lionel JOSPIN,Parti Socialiste (PS),Gauche,735549.0,805794.0,665278.0,...,-4387.26,7.9,17.6,37.2,25.893413,28.366236,23.419674,15.810440,6.510237,0
2,2002,1811287.0,83.03,88.56,Jacques CHIRAC,Rassemblement pour la République (RPR),Droite républicaine,738672.0,783368.0,777030.0,...,7618.00,11.5,17.7,32.4,24.772978,26.271956,26.059398,14.512411,8.383256,1
3,2002,1811287.0,83.03,11.44,Jean-Marie LE PEN,Front National (FN),Extrême droite,738672.0,783368.0,777030.0,...,7618.00,11.5,17.7,32.4,24.772978,26.271956,26.059398,14.512411,8.383256,0
4,2007,2025199.0,87.53,47.38,Nicolas SARKOZY,Union pour un Mouvement Populaire (UMP),Droite républicaine,762853.0,771058.0,856770.0,...,12146.00,12.6,16.4,33.7,24.448160,24.711116,27.458042,13.977235,9.405446,0


In [ ]:
# prompt: puedes explicarme los resultados del entrenamiento:
# 4. Entraînement du modèle de régression XGBoost
# Résultats du modèle de régression (prédiction du % de voix):
# MAE: 28.27
# RMSE: 34.89
# R²: -3.08

Okay, let's analyze the XGBoost regression training results you provided:

**Results:**

* **MAE (Mean Absolute Error): 28.27**
* **RMSE (Root Mean Squared Error): 34.89**
* **R² (R-squared): -3.08**


**Interpretation:**

1. **MAE and RMSE:** These metrics represent the average prediction error of your model. Lower values are better.
   * **MAE** tells you the average absolute difference between your model's predictions and the actual values. In your case, it's 28.27, meaning on average, your model's predictions for the percentage of votes are off by about 28.27 percentage points.
   * **RMSE** gives you a measure of the standard deviation of the residuals (the differences between predicted and actual values). It's similar to MAE but penalizes larger errors more heavily. Your RMSE of 34.89 indicates a relatively large average error.

2. **R² (R-squared):** This metric represents the proportion of the variance in the dependent variable (percentage of votes) that's explained by the independent variables (features) in your model. A higher R² indicates a better fit.
   * Your R² is **-3.08**. This is a very unusual and concerning result. A negative R² suggests that your model is performing worse than simply predicting the mean value of the target variable.

**Possible Reasons for Negative R²:**

* **Model Complexity:** Your model might be overly complex and overfitting the training data. This can lead to poor generalization to unseen data and a negative R².
* **Feature Selection:** The features you've chosen might not be relevant to predicting the percentage of votes or might be highly correlated with each other, leading to multicollinearity issues.
* **Data Issues:** There might be outliers, missing values, or inconsistencies in your dataset that are negatively impacting the model's performance.
* **Incorrect Hyperparameters:** The hyperparameters you've chosen for your XGBoost model (like the number of estimators, learning rate, etc.) might not be optimal for your dataset.
* **Inappropriate Model Choice:** It's possible that a different regression model (like linear regression, random forest regression) might be better suited for your data.

**Recommendations:**

* **Check for Overfitting:** Evaluate your model's performance on a validation set (or use cross-validation) to ensure it's not overfitting the training data.
* **Improve Feature Selection:** Carefully examine the features you're using. You might need to explore feature engineering to create new variables that are more predictive. Consider removing irrelevant or redundant features.
* **Analyze Data:** Thoroughly examine your dataset for potential issues like missing data, outliers, or incorrect data entries.
* **Tune Hyperparameters:** Experiment with different hyperparameter settings for your XGBoost model to improve its performance. You can use techniques like grid search or random search to find optimal settings.
* **Consider Different Regression Models:** Explore using other regression models like linear regression or random forest regression to see if they offer a better fit for your data.

**Further Steps:**

1. **Visualize Predictions and Residuals:** Examine how well your model's predictions align with the actual values by creating scatter plots. Analyze the residuals (difference between predicted and actual values) to understand if there are systematic patterns in your model's errors.
2. **Explore Feature Importance:** Examine the feature importance scores from your XGBoost model. This can help identify which variables are most influential in making predictions.
3. **Consider Regularization:** Techniques like L1 or L2 regularization can help prevent overfitting and improve model generalization.



By investigating these aspects, you'll be able to diagnose the problem and make improvements to your model to achieve more accurate and meaningful results. I hope this helps! Let me know if you have further questions.